In [ ]:
## Collecte de données

Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :

1.  Créer un dossier appelé *images*.
2.  Télécharger les images sous licence ouverte dans le dossier *images* (minimum 100
    images).
3.  Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image (.jpeg,
    .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.),
    date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations [Exif](https://en.wikipedia.org/wiki/Exif) présentes dans les fichiers d'images.

In [ ]:
import os

# Spécifiez le chemin du dossier que vous souhaitez créer
dossier = 'images'

# Créez le dossier
os.makedirs(dossier, exist_ok=True)

print(f"Le dossier '{dossier}' a été créé avec succès.")


In [ ]:
!pip install SPARQLWrapper

In [ ]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Initialiser le wrapper SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Définir la requête SPARQL pour obtenir des images de monuments
sparql.setQuery("""
SELECT ?monument ?monumentLabel ?image WHERE {
  ?monument wdt:P31 wd:Q839954.  # Monument
  ?monument wdt:P18 ?image.      # Image
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)

try:
    # Exécuter la requête et récupérer les résultats en JSON
    results = sparql.query().convert()
except Exception as e:
    print(f"❌ Erreur lors de l'exécution de la requête SPARQL : {e}")
    exit(1)

# Liste des images à télécharger avec des indices de 1 à 100
images = [(i + 1, result["image"]["value"]) for i, result in enumerate(results["results"]["bindings"])]

# Définition d'un User-Agent personnalisé
HEADERS = {
    "User-Agent": "MonScript/1.0 (mailto:mohamedguef@gmail.com)"  # Remplace par ton email pour respecter la politique
}

def download_image(data):
    index, image_url = data
    try:
        # Nom fixe sous le format image_1.jpg, image_2.jpg, ..., image_100.jpg
        image_name = f"image_{index}.jpg"
        image_path = os.path.join(IMAGE_DIR, image_name)

        # Télécharger l'image avec un User-Agent correct
        response = requests.get(image_url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Vérifie si la requête a réussi
        
        # Sauvegarder l'image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ Téléchargé : {image_name}")

    except requests.RequestException as e:
        print(f"⚠️ Erreur lors du téléchargement de {image_url} : {e}")
    except Exception as e:
        print(f"⚠️ Erreur inattendue : {e}")

# Téléchargement des images en parallèle
MAX_THREADS = 10  # Ajuste ce nombre en fonction de ta connexion et des performances

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, images)

print("✅ Tous les téléchargements sont terminés !")


In [ ]:
!pip install Pillow ExifRead

In [ ]:
!pip install tqdm

In [ ]:
import os
import json
from PIL import Image, UnidentifiedImageError
import exifread
from tqdm import tqdm  # Barre de progression

# Définir le dossier des images et le fichier de sortie JSON
IMAGE_DIR = "images"
METADATA_FILE = os.path.join(IMAGE_DIR, "metadata.json")

# Vérifier que le dossier "images" existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable. Vérifiez que les images sont bien téléchargées.")
    exit(1)

# Liste pour stocker les métadonnées de toutes les images
metadata_list = []

# Liste des fichiers images triés
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Parcourir les images et extraire les métadonnées
for image_name in tqdm(image_files, desc="📷 Extraction des métadonnées"):
    image_path = os.path.join(IMAGE_DIR, image_name)

    try:
        # Vérifier si l'image est corrompue en tentant de l'ouvrir
        with Image.open(image_path) as img:
            img.verify()  # Vérifie l'intégrité de l'image sans la charger
            img = Image.open(image_path)  # Recharge l'image pour la lecture des métadonnées
            width, height = img.size
            format_ = img.format
            orientation = 'Portrait' if height > width else 'Paysage' if width > height else 'Carré'

        # Vérifier si c'est un PNG (pas d'Exif)
        if format_.upper() == "PNG":
            exif_data = "Non disponible (format PNG)"
        else:
            # Extraire les métadonnées Exif avec exifread
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal", details=False)

            # Filtrer les champs problématiques
            exif_data = {
                "Date de création": str(tags.get("EXIF DateTimeOriginal", "Inconnu")),
                "Modèle d'appareil": str(tags.get("Image Model", "Inconnu")),
                "Marque d'appareil": str(tags.get("Image Make", "Inconnu")),
                "ISO": str(tags.get("EXIF ISOSpeedRatings", "Inconnu")),
                "Temps d'exposition": str(tags.get("EXIF ExposureTime", "Inconnu")),
                "Ouverture (f)": str(tags.get("EXIF FNumber", "Inconnu")),
                "Longueur focale": str(tags.get("EXIF FocalLength", "Inconnu")),
                "GPS Latitude": str(tags.get("GPS GPSLatitude", "Non disponible")),
                "GPS Longitude": str(tags.get("GPS GPSLongitude", "Non disponible")),
            }

            # Supprimer les champs corrompus (évite les erreurs "Possibly corrupted field")
            exif_data = {key: value for key, value in exif_data.items() if "Possibly corrupted" not in value}

        # Ajouter les métadonnées à la liste
        metadata_list.append({
            'Nom du fichier': image_name,
            'Taille': {'Largeur': width, 'Hauteur': height},
            'Format': format_,
            'Orientation': orientation,
            'Exif': exif_data
        })

    except UnidentifiedImageError:
        print(f"⚠️ Image illisible : {image_name} (corrompue ou format inconnu)")
    except Exception as e:
        print(f"⚠️ Erreur inattendue sur {image_name} : {e}")

# Enregistrer toutes les métadonnées dans un fichier JSON
try:
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"✅ Métadonnées enregistrées dans '{METADATA_FILE}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")


In [1]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import json

# Fonction pour extraire les couleurs dominantes
def get_dominant_colors(image_path, k=3):
    """
    Extrait les k couleurs dominantes d'une image en utilisant K-Means.
    :param image_path: Chemin de l'image
    :param k: Nombre de couleurs dominantes à extraire
    :return: Liste des couleurs dominantes au format RGB
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Impossible de lire l'image : {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB
    pixels = image.reshape(-1, 3)  # Redimensionner en tableau 2D (N pixels x 3 canaux)
    kmeans = KMeans(n_clusters=k)  # Appliquer K-Means
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)  # Extraire les couleurs dominantes
    return colors.tolist()  # Convertir en liste pour JSON

# Dossier contenant les images
IMAGE_DIR = "images"

# Vérifier que le dossier existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable.")
    exit(1)

# Dictionnaire pour stocker les couleurs dominantes de chaque image
dominant_colors_data = {}

# Parcourir toutes les images du dossier
for image_name in os.listdir(IMAGE_DIR):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers images
        image_path = os.path.join(IMAGE_DIR, image_name)
        print(f"🔍 Traitement de l'image : {image_name}")

        # Extraire les couleurs dominantes
        dominant_colors = get_dominant_colors(image_path, k=3)  # Extraire 3 couleurs dominantes
        if dominant_colors:
            dominant_colors_data[image_name] = dominant_colors
            print(f"✅ Couleurs dominantes pour {image_name} : {dominant_colors}")
        else:
            print(f"⚠️ Aucune couleur dominante trouvée pour {image_name}")

# Enregistrer les résultats dans un fichier JSON
output_file = os.path.join(IMAGE_DIR, "dominant_colors.json")
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dominant_colors_data, f, indent=4, ensure_ascii=False)
    print(f"✅ Couleurs dominantes enregistrées dans '{output_file}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")

🔍 Traitement de l'image : image_1.jpg
✅ Couleurs dominantes pour image_1.jpg : [[130, 135, 102], [50, 58, 45], [191, 198, 201]]
🔍 Traitement de l'image : image_10.jpg
✅ Couleurs dominantes pour image_10.jpg : [[53, 64, 46], [192, 202, 186], [119, 138, 107]]
🔍 Traitement de l'image : image_100.jpg
✅ Couleurs dominantes pour image_100.jpg : [[80, 36, 34], [218, 202, 167], [157, 128, 107]]
🔍 Traitement de l'image : image_11.jpg
✅ Couleurs dominantes pour image_11.jpg : [[125, 151, 173], [116, 94, 72], [229, 217, 203]]
🔍 Traitement de l'image : image_12.jpg
✅ Couleurs dominantes pour image_12.jpg : [[103, 112, 123], [229, 195, 155], [56, 39, 34]]
🔍 Traitement de l'image : image_13.jpg
✅ Couleurs dominantes pour image_13.jpg : [[174, 147, 92], [102, 120, 107], [207, 183, 121]]
🔍 Traitement de l'image : image_14.jpg
✅ Couleurs dominantes pour image_14.jpg : [[203, 181, 126], [105, 119, 95], [175, 149, 86]]
🔍 Traitement de l'image : image_15.jpg
✅ Couleurs dominantes pour image_15.jpg : [[195